# Problem Statement
### AtliQ Motors is an automotive giant from the USA specializing in electric vehicles (EV). In the last 5 years, their market share rose to 25% in electric and hybrid vehicles segment in North America.

### As a part of their expansion plans, they wanted to launch their bestselling models in India where their market share is less than 2%. Bruce Haryali, the chief of AtliQ Motors India wanted to do a detailed market study of existing EV/Hybrid market in India before proceeding further. Bruce gave this task to the data analytics team of AtliQ motors and Peter Pandey is the data analyst working in this team. 

- Expansion
- Market Study


In [1]:
# What's happening in your market or industery?
# Who are your competitors?
# What cusotmers do you serve?
# How are we performing?

In [2]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf as run

In [3]:
makers = pd.read_csv(r"C:\Users\m\Taha\Codebasics Challenges\EV\RPC12_Input_For_Participants\datasets\electric_vehicle_sales_by_makers.csv")
date = pd.read_csv(r"C:\Users\m\Taha\Codebasics Challenges\EV\RPC12_Input_For_Participants\datasets\dim_date.csv")


# Makers

In [4]:
makers.head()

,date,vehicle_category,maker,electric_vehicles_sold
0,01-Apr-21,2-Wheelers,OLA ELECTRIC,0
1,01-Apr-22,2-Wheelers,OKAYA EV,0
2,01-May-21,2-Wheelers,OLA ELECTRIC,0
3,01-Jun-21,2-Wheelers,OLA ELECTRIC,0
4,01-Jul-21,2-Wheelers,OLA ELECTRIC,0


In [5]:
date.head()

,date,fiscal_year,quarter
0,01-Apr-21,2022,Q1
1,01-May-21,2022,Q1
2,01-Jun-21,2022,Q1
3,01-Jul-21,2022,Q2
4,01-Aug-21,2022,Q2


In [6]:
makers['date'] = pd.to_datetime(makers['date'], dayfirst = True)
date['date'] = pd.to_datetime(date['date'], dayfirst = True)

C:\Users\m\AppData\Local\Temp\ipykernel_21920\158231520.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  makers['date'] = pd.to_datetime(makers['date'], dayfirst = True)
C:\Users\m\AppData\Local\Temp\ipykernel_21920\158231520.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date['date'] = pd.to_datetime(date['date'], dayfirst = True)


In [7]:
df = pd.merge(makers,date, on = 'date')
df.head()

,date,vehicle_category,maker,electric_vehicles_sold,fiscal_year,quarter
0,2021-04-01,2-Wheelers,OLA ELECTRIC,0,2022,Q1
1,2021-04-01,4-Wheelers,BYD India,0,2022,Q1
2,2021-04-01,4-Wheelers,PCA Automobiles,0,2022,Q1
3,2021-04-01,4-Wheelers,BMW India,0,2022,Q1
4,2021-04-01,4-Wheelers,Volvo Auto India,0,2022,Q1


In [8]:
df.shape

(816, 6)

In [9]:
df.nunique()

date                       36
vehicle_category            2
maker                      26
electric_vehicles_sold    603
fiscal_year                 3
quarter                     4
dtype: int64

In [10]:
df.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
811    False
812    False
813    False
814    False
815    False
Length: 816, dtype: bool

In [11]:
df['maker'] = df['maker'].str.strip().str.title()

In [12]:
df.rename(columns = {'electric_vehicles_sold' : 'EV'}, inplace = True)

# Who are your competitors?

In [13]:
makers['maker'].unique()

array(['OLA ELECTRIC', 'OKAYA EV', 'BYD India', 'PCA Automobiles',
       'BMW India', 'Volvo Auto India', 'KIA Motors', 'Mercedes -Benz AG',
       'Tata Motors', 'MG Motor', 'Mahindra & Mahindra', 'Hyundai Motor',
       'HERO ELECTRIC', 'OKINAWA', 'AMPERE', 'ATHER', 'PURE EV', 'TVS',
       'REVOLT', 'BAJAJ', 'BEING', 'JITENDRA', 'OTHERS', 'BGAUSS',
       'BATTRE ELECTRIC', 'KINETIC GREEN'], dtype=object)

In [14]:
df_sales = df.pivot_table(index = 'maker',
                   values = 'EV',
                   aggfunc = 'sum').sort_values(by = 'EV', ascending = False)

# Top 5 Competitors

In [15]:
df_sales.head()

,EV
maker,
Ola Electric,489473
Tvs,272575
Ather,204449
Hero Electric,170394
Ampere,167274


# Bottom 5 Competitors

In [16]:
df_sales.tail()

,EV
maker,
Pca Automobiles,1684
Bmw India,1370
Volvo Auto India,568
Kia Motors,557
Mercedes -Benz Ag,388


In [29]:
df_sales.describe()

,EV
count,26.000000
mean,79465.807692
std,115646.047641
min,388.000000
25%,3024.500000
50%,22458.000000
75%,153754.750000
max,489473.000000


In [33]:
df.groupby(['fiscal_year', 'maker'])['EV'].sum().reset_index().groupby('fiscal_year')['EV'].mean()

fiscal_year
2022    12325.000000
2023    33711.652174
2024    44330.130435
Name: EV, dtype: float64

# Market Share

In [18]:
query = """
with mk_share as
    (select
        maker,
        fiscal_year,
        sum(EV) as EV,
        round((cast(sum(EV) as real) / (select sum(EV) from df)) * 100, 2) as market_share
    from
        df
    group by
        maker, fiscal_year
    order by
        market_share desc),
filter as
    (select
        *,
        row_number() over(partition by fiscal_year order by market_share desc) as rnk
    from 
        mk_share)
select
    *
from
    filter
"""
mk_share = run(query)
mk_share

,maker,fiscal_year,EV,market_share,rnk
0,Hero Electric,2022,69452,3.36,1
1,Okinawa,2022,47665,2.31,2
2,Ampere,2022,25510,1.23,3
3,Others,2022,24858,1.20,4
4,Ather,2022,19976,0.97,5
...,...,...,...,...,...
63,Pca Automobiles,2024,1533,0.07,19
64,Bmw India,2024,1078,0.05,20
65,Kia Motors,2024,328,0.02,21
66,Volvo Auto India,2024,459,0.02,22


In [19]:
mk_share[mk_share['rnk'].isin([1,2,3,4,5])]

,maker,fiscal_year,EV,market_share,rnk
0,Hero Electric,2022,69452,3.36,1
1,Okinawa,2022,47665,2.31,2
2,Ampere,2022,25510,1.23,3
3,Others,2022,24858,1.20,4
4,Ather,2022,19976,0.97,5
22,Ola Electric,2023,152583,7.39,1
23,Okinawa,2023,96945,4.69,2
24,Hero Electric,2023,88993,4.31,3
25,Ampere,2023,87376,4.23,4
26,Tvs,2023,82093,3.97,5


In [20]:
mk_share.pivot_table(index = 'maker',
                    columns = 'fiscal_year',
                    values = 'rnk')

fiscal_year,2022,2023,2024
maker,,,
Ampere,3.0,4.0,6.0
Ather,5.0,6.0,3.0
Bajaj,12.0,8.0,4.0
Battre Electric,NaN,NaN,16.0
Being,11.0,14.0,NaN
Bgauss,NaN,NaN,10.0
Bmw India,17.0,19.0,20.0
Byd India,18.0,17.0,17.0
Hero Electric,1.0,3.0,12.0


In [26]:
query = """
select
    df.maker,
    sum(df.EV) as EV_2022,
    sum(df1.EV) as EV_2024
from
    df
join
    df as df1
on
    df.maker = df1.maker
where
    df.fiscal_year = 2022 and
    df1.fiscal_year = 2024 and
    df.EV <> 0
group by
    df.maker
"""
makers_CAGR = run(query)
makers_CAGR

,maker,EV_2022,EV_2024
0,Ampere,306120,652656
1,Ather,239712,1290624
2,Bajaj,85368,1268340
3,Bmw India,84,1078
4,Byd India,396,5864
5,Hero Electric,833424,143388
6,Hyundai Motor,1320,16680
7,Mahindra & Mahindra,48504,280152
8,Mercedes -Benz Ag,312,3201
9,Mg Motor,19764,105948


In [27]:
makers_CAGR['CAGR'] = round((((makers_CAGR['EV_2024']/makers_CAGR['EV_2022'])**(1/3)) - 1) * 100,2)

In [28]:
makers_CAGR.sort_values(by = 'CAGR', ascending = False)

,maker,EV_2022,EV_2024,CAGR
15,Tvs,116868,2168916,164.76
2,Bajaj,85368,1268340,145.84
4,Byd India,396,5864,145.56
3,Bmw India,84,1078,134.12
6,Hyundai Motor,1320,16680,132.92
8,Mercedes -Benz Ag,312,3201,117.29
16,Volvo Auto India,48,459,112.25
11,Ola Electric,172812,1289956,95.43
7,Mahindra & Mahindra,48504,280152,79.42
1,Ather,239712,1290624,75.27
